In [4]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from scipy.spatial.distance import cdist

# lineare klassifikation

In [3]:
def e_in(X, y, w):
    return np.sum(H(X, w) != y) / X.shape[0]

# phi (x -> {0, 1})
def phi(x):
    return np.sign(x)

# perzeptron (x, w -> y)
def h(x, w):
    return phi(w @ x)

def H(X, w):
    return [h(x, w) for x in X]

# p(x) (x, w -> gerade)
def p(x, w):
    return -w[1]/w[2]*x - w[0]/w[2]

def P(X, w):
    return [p(x, w) for x in X]

# pla (X, y -> w)
def pla(X, y, w=None, t=0):
    if t == 0:
        X = np.column_stack((np.ones(X.shape[0]), X))
    if w is None:
        w = np.zeros(X.shape[1])

    for i, x in enumerate(X):
        if h(x, w) != y[i]:
            return pla(X, y, w + y[i]*x, t+1)

    return w

# pocket (X, y -> w)
def pocket(X, y, T):
    X = np.column_stack((np.ones(X.shape[0]), X))

    w = np.zeros(X.shape[1])
    w_d = w.copy()
    e = 1

    for _ in range(T):
        for i, x in enumerate(X):
            if h(x, w) != y[i]:
                w += y[i]*x

                if e_in(X, y, w) < e:
                    w_d = w.copy()
                    e = e_in(X, y, w_d)

    return w_d

def scatter(X, y, w):
    xlim = (X[:,0].min()-0.1, X[:,0].max()+0.1)
    ylim = (X[:,1].min()-0.1, X[:,1].max()+0.1)

    plt.scatter(X[:,0], X[:,1], c=y)
    plt.plot(xlim, P(xlim, w))
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

# lineare regression

In [6]:
def e_in(X, y, w, Q=4):
    g = G(X, w, Q)

    return float((g-y).T@(g-y) / X.shape[0])

def e_out(f, w, start=-1, stop=1, K=50):
    L = np.linspace(start, stop, K)
    
    return e_in(L, f(L), w)
    
# g ~ f (x, w -> y)
def g(x, w):
    return np.hstack((np.ones(1), x)) @ w

def G(X, w):
    return [g(x, w) for x in X]

# lineare regression (X, y -> w)
def lin_reg(X, y):
    X = np.column_stack((np.ones(X.shape[0]), X))

    X_d = np.linalg.inv(X.T @ X) @ X.T
    return X_d @ y

def scatter(X, y, w):
    xlim = (X.min()-0.1, X.max()+0.1)
    ylim = (y.min()-0.1, y.max()+0.1)

    plt.xlabel("X")
    plt.ylabel("y")

    plt.legend(handles=[
        mpatches.Patch(color="#1f77b4", label="f(x)"),
        mpatches.Patch(color="#ff7f0e", label="g(x)")
    ])

    plt.scatter(X, y, c="#1f77b4")
    plt.plot(xlim, G(xlim, w), c="#ff7f0e")
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [7]:
# lineare regression mit featuretransformation

# featuretranformation (X -> Z)
def phi(x, Q=4):
    return np.array([x**i for i in range(1, Q+1)])

def PHI(X, Q=4):
    return np.concatenate([phi(x, Q).reshape(1, -1) for x in X])

# g ~ f (x -> z; z, w -> y)
def g(x, w, Q=4):
    return np.hstack((np.ones(1), phi(x, Q))) @ w

def G(X, w, Q=4):
    return [g(x, w, Q) for x in X]

# lineare regresseion (X -> Z; Z, y -> w)
def lin_reg_featuretransformed(X, y, Q=4):
    return lin_reg(PHI(X, Q), y)
    
def scatter(X, y, w, f=None, Q=4, start=None, stop=None, K=50):
    if start is None or stop is None:
        L = np.linspace(X.min(), X.max(), K)
    else:
        L = np.linspace(start, stop, K)

    plt.xlabel("X")
    plt.ylabel("y")

    plt.legend(handles=[
        mpatches.Patch(color="#1f77b4", label="f(x)"),
        mpatches.Patch(color="#ff7f0e", label="g(x)")
    ])

    plt.scatter(X, y, c="#1f77b4")
    if f is not None: plt.plot(L, f(L), c="#1f77b4")
    plt.plot(L, G(L, w, Q), c="#ff7f0e")
    plt.show()

In [8]:
# lineare regression mit regularisierung

# lineare regression (X, y -> w)
def lin_reg_regularized(X, y, l=0):
    X = np.column_stack((np.ones(X.shape[0]), X))

    X_d = np.linalg.inv(X.T @ X + l * np.identity(np.min(X.shape))) @ X.T
    return X_d @ y

In [9]:
# lineare regression mit featuretransformation und regularisierung

# lineare regression (X -> Z; Z, y -> w)
def lin_reg_featuretransformed_regularized(X, y, Q=4, l=0):
    return lin_reg_regularized(PHI(X, Q), l)

# logistische regression

In [5]:
def e_in(X, y, w):
    return np.sum([np.log(1 + np.exp(-y[i] * w @ x)) for i, x in enumerate(X)]) / X.shape[0]

# phi (x -> [0, 1])
def phi(x):
    return 1 / (1 + np.exp(-x))

# gradient (X, y, w -> E_in)
def gradient(X, y, w):
    return np.sum([(-y[i]*x) * phi(-y[i] * w @ x) for i, x in enumerate(X)], axis=0) / X.shape[0]

# gradientenabstieg (X, y, w_0 -> w)
def grad_descent(X, y, w=None, n=0.1, T=400, eps_e=10**-8, eps_g=10**-8):
    if w is None:
        w = np.random.normal(0, 1, X.shape[1])

    for _ in range(T):
        g = gradient(X, y, w)
        v = -g / np.linalg.norm(g)

        w = w + n*v

        if e_in(X, y, w) < eps_e: break
        if np.linalg.norm(g) < eps_g: break
    
    return w

# entscheidungsbäume

In [4]:
# entropie
def entropy(X):
    def xlogx(x):
        if x == 0: return 0
        return x * np.log2(x)
    
    return -np.sum([xlogx(sum([x == l for l in X]) / len(X)) for x in np.unique(X)])

# information gain
def info_gain(y, x_j, z):
    i = x_j < z
    return entropy(y) - np.sum(i) / len(y) * entropy(y[i]) - np.sum(~i) / len(y) * entropy(y[~i])

def max_info_gain(y, X):
    IG = 0
    ret = [None, None]

    for j in range(X.shape[1]):
        for z in np.unique(X[:,j]):
            if info_gain(y, X[:,j], z) > IG:
                IG = info_gain(y, X[:,j], z)
                ret = [j, z]

    return ret

# entscheidungsbaum
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y, depth=0):
        def make_node(feature=None, split=None, left=None, right=None):
            values, counts = np.unique(y, return_counts=True)

            return {
                'depth': depth,
                'feature': feature,
                'split': split,
                'value': values[np.argmax(counts)],
                'left': left,
                'right': right
            }

        if len(y) == 0: return None
        if depth >= self.max_depth or len(np.unique(y)) == 1: return make_node()
            
        i, z = max_info_gain(y, X)
        if z is None: return make_node()

        idx = np.array([x[i] < z for x in X])
        node = make_node(i, z, self.fit(X[idx], y[idx], depth+1), self.fit(X[~idx], y[~idx], depth+1))
        
        if depth == 0:
            self.tree = node
            return self
            
        return node
            

    def predict(self, X):
        def rek_predict(x, ptr):
            if ptr['split'] is None: return ptr['value']
            if x[ptr['feature']] < ptr['split']: return rek_predict(x, ptr['left'])
            return rek_predict(x, ptr['right'])
            
        return [rek_predict(x, self.tree) for x in X]

# LS-SVM

In [ ]:
def kernel_rbf(X, X_, sigma):
    return np.exp(-cdist(X, X_, metric="euclidean")**2 / (2 * sigma**2))

def omega(X, y, sigma):
    return (y @ y.T) * kernel_rbf(X, X, sigma)
    
def lssvm(X, y, sigma, gamma):
    A = np.block([
        [0, y.T],
        [y, omega(X, y, sigma) + np.identity(X.shape[0]) / gamma]
    ])
    z = np.concatenate(([0], np.ones(X.shape[0])))
    
    s = (np.linalg.inv(A) @ z)

    return s[1:], s[0]

# bonus

$$\begin{align*}
& H(X) && = - \sum_{k=1}^C p_k \log_2(p_k) \\
& H(Y|X) && = - \sum_j p(x_j) \left(\sum_i p(y_i|x_j) \log_2 \left(p(y_i|x_j)\right)\right) \\
& IG(Y,X) && = H(Y) - H(Y|X) \\
\end{align*}$$

In [2]:
# entropien und information gain
def H(X, y=None, tex=False):
    def xlogx(x):
        if x == 0: return 0
        return x * np.log2(x)
    
    if tex: print("= ", end="")

    if y is None:
        h = 0
        for x in set(X):
            if tex:
                frac_p_k = f"\\frac{{{sum([x == l for l in X])}}}{{{len(X)}}}"
                print(f"- {frac_p_k} \\log_2\\left({frac_p_k}\\right) ", end="")

            h -= xlogx(sum([x == l for l in X]) / len(X))
    else:
        h = 0
        for x in set(X):
            idx = [i for i, l in enumerate(X) if x == l]
            p_yx = [y[i] for i in idx]
            
            if tex:
                frac_p_x = f"\\frac{{{len(idx)}}}{{{len(X)}}}"
                print(f"- {frac_p_x} \\left(", end="")

                frac_p_yx = f"\\frac{{{sum(p_yx)}}}{{{len(p_yx)}}}"
                print(f"{frac_p_yx} \\log_2\\left({frac_p_yx}\\right) ", end="")
                frac_p_yx = f"\\frac{{{len(p_yx)-sum(p_yx)}}}{{{len(p_yx)}}}"
                print(f"+ {frac_p_yx} \\log_2\\left({frac_p_yx}\\right) ", end="")

                print(f"\\right) ", end="")

            p_yx = sum(p_yx) / len(p_yx)

            h -= len(idx) / len(X) * (xlogx(p_yx) + xlogx(1 - p_yx))
    return h

def IG(X, y):
    return H(y) - H(X, y)

def ig_to_tex(X, y, labels=None):
    print("$$\\begin{align*}")
    print("\\\\")

    print(f"& H(X) && = - \\sum_{{k=1}}^C p_k \\log_2(p_k) & \\\\")
    print(f"& H(Y|X) && = - \\sum_j p(x_j) \\left(\\sum_i p(y_i|x_j) \\log_2 \\left(p(y_i|x_j)\\right)\\right) & \\\\")
    print(f"& IG(Y,X) && = H(Y) - H(Y|X) &  \\\\")
    print("\\\\")

    print(f"& H(Y) = \\mathbb{{E}}[i] && ", end="")
    print(f"& \\approx {H(y, tex=True):.5f} \\\\")
    print("\\\\")

    for i in range(len(X)):
        label = f"\\text{{{labels[i]}}}" if labels is not None else f"\\text{{X[{i}]}}"
        
        print(f"& H(Y|{label}) && ", end="")
        print(f"& \\approx {H(X[i], y, tex=True):.5f} \\\\")
        
        print(f"& IG(Y,{label}) && = H(Y) - H(Y|{label}) & \\approx {IG(X[i], y):.5f} \\\\")
        print("\\\\")
    
    print("\\end{align*}$$")